In [26]:
import Pkg; Pkg.add("CSV")

┌ Warning: could not download https://pkg.julialang.org/registries
│   exception = Downloads.RequestError("https://pkg.julialang.org/registries", 28, "Resolving timed out after 30001 milliseconds", Downloads.Response(nothing, "https://pkg.julialang.org/registries", 0, "", Pair{String, String}[]))
└ @ Pkg.Registry /Users/julia/.julia/scratchspaces/a66863c6-20e8-4ff4-8a62-49f30b1f605e/agent-cache/default-macmini-x64-5.0/build/default-macmini-x64-5-0/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Pkg/src/Registry/Registry.jl:68
    Updating registry at `~/.julia/registries/General.toml`
   Resolving

 package versions...


   Installed InlineStrings ─── v1.4.0
   Installed WorkerUtilities ─ v1.6.1
   Installed SentinelArrays ── v1.3.18
   Installed WeakRefStrings ── v1.4.2
   Installed PooledArrays ──── v1.4.2
   Installed FilePathsBase ─── v0.9.20


   Installed CSV ───────────── v0.10.9


    Updating `~/.julia/environments/v1.8/Project.toml`
  [336ed68f] + CSV v0.10.9
    Updating `~/.julia/environments/v1.8/Manifest.toml`


...

  ✓ SentinelArrays


  ✓ FilePathsBase


  ✓ InlineStrings


  ✓ WeakRefStrings


  ✓ CSV
  7 dependencies successfully precompiled in 98 seconds. 176 already precompiled.


In [3]:
import Pkg; Pkg.add("DataFrames")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


   Installed InvertedIndices ──── v1.3.0


   Installed Crayons ──────────── v4.1.1


   Installed StringManipulation ─ v0.3.0


...

  ✓ Crayons


  ✓ StringManipulation


  ✓ PrettyTables


  ✓ DataFrames


  5 dependencies successfully precompiled in 193 seconds. 183 already precompiled.


In [5]:
 # Credit to Chatgpt and collboration with Mica
 using CSV
 using DataFrames

 nodes=CSV.read("names.csv", DataFrames.DataFrame)
 edges=CSV.read("edges.csv", DataFrames.DataFrame)

nodes=Matrix(nodes)
edges=Matrix(edges)



# function page_rank(nodes, edges, N)
#     A = adjacency_matrix(nodes, edges)
#     T = transition_matrix(A)
#     n = size(A)[1]
#     x = ones(n) / n
#     for i in 1:N
#         x = T * x
#     end
#     return x
# end

# function transition_matrix(A)
#     n = size(A)[1]
#     T = zeros(n, n)
#     for i = 1:n
#         s = sum(A[:, i])
#         if s == 0
#             T[:, i] = ones(n) / n
#         else
#             T[:, i] = A[:, i] / s
#         end
#     end
#     return T
# end

# function adjacency_matrix(nodes, edges)
#     n = size(nodes, 1)
#     A = zeros(n, n)
#     for i = 1:size(edges, 1)
#         u, v = edges[i, :]
#         A[v, u] = 1
#     end
#     return A
# end


# # run PageRank
# N = 10
# x = page_rank(nodes, edges, N)
# println(x)




using SparseArrays 
using LinearAlgebra 

n = size(nodes, 1)

# 创建稀疏的邻接矩阵
A = sparse(
    edges[2:end, 1],   # row indices
    edges[2:end, 2],   # column indices
    fill(1.0, size(edges, 1)-1),  # edge weights
    n,             # number of rows
    n              # number of columns
)
A = A'

# 创建转移矩阵T
row_sums = vec(sum(A, dims=2))
probs = ifelse.(row_sums .== 0, 0.0, 1 ./ row_sums)
diag_matrix = spdiagm(0 => probs)
T = A' * diag_matrix

# 使用幂迭代计算PageRank值
function power_iteration(A, T, x)
    eps = 1e-15  # 容忍误差
    max_iter = 1000  # 最大迭代次数
    iter = 0
    
    while iter < max_iter
        x_new = T * x
        if norm(A * diag_matrix * x_new - x_new) < eps
            break
        end
        
        x = x_new
        iter += 1
    end
    
    return x
end

# 计算页面的PageRank值
x = ones(n)/n
pagerank = power_iteration(A, T, x)

# 按PageRank值对页面进行排序
ranks = sortperm(pagerank, rev=true)

# 输出前10个页面的名称和PageRank值
for i in 1:min(10, n)
    page_idx = ranks[i]
    page_name = nodes[page_idx, 1]
    page_rank = pagerank[page_idx]
    println("Rank $i: $page_name - $page_rank")
end







Rank 1: Outline of geography - 0.00029697495488331265
Rank 2: 103rd United States Congress - 0.00027225298582670066
Rank 3: France - 0.0002446745973632674
Rank 4: Index of physics articles - 0.00024165089322736956
Rank 5: 87th United States Congress - 0.0002385516577487926
Rank 6: 2010 United States Census - 0.0002379918783749957
Rank 7: 82nd United States Congress - 0.00023656956713882297
Rank 8: United Kingdom - 0.00022103737528475397
Rank 9: Physics - 0.00021376762048081084
Rank 10: List of stadiums by capacity - 0.0002094233799784
